In [16]:
dict1 = {1:"a", 2:"b"}
print(dict1)
s = list(dict1.items())
print(s)
for i in s:
    print(i[0], i[1])

{1: 'a', 2: 'b'}
[(1, 'a'), (2, 'b')]
1 a
2 b


In [17]:
dict1[3]= "b"
print(dict1)
dict2 = {4:"d"}

{1: 'a', 2: 'b', 3: 'b'}


In [19]:
dict1.update(dict2)
print(dict1)

{1: 'a', 2: 'b', 3: 'b', 4: 'd'}


In [1]:
import pandas as pd

data = [{'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'Completed'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable1'}, 'file_name_time': {'S': 'file1_time1'}}, {'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'inprogress'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable2'}, 'file_name_time': {'S': 'file2_time2'}}, {'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'Fail'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable3'}, 'file_name_time': {'S': 'file3_time3'}}, {'raw_ingestion_status': {'S': 'Pending'}, 'inhance_injection': {'S': 'Fail'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable4'}, 'file_name_time': {'S': 'file3_time4'}}]

formatted_data = []

for entry in data:
    formatted_entry = {}
    for key, value in entry.items():
        formatted_entry[key] = value['S']
    formatted_data.append(formatted_entry)

df = pd.DataFrame(formatted_data)

print(df)


  raw_ingestion_status inhance_injection source_system  status    table_name  \
0            Processed         Completed          alfa  Active  SampleTable1   
1            Processed        inprogress          alfa  Active  SampleTable2   
2            Processed              Fail          alfa  Active  SampleTable3   
3              Pending              Fail          alfa  Active  SampleTable4   

  file_name_time  
0    file1_time1  
1    file2_time2  
2    file3_time3  
3    file3_time4  


In [11]:
data = [{'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'Completed'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable1'}, 'file_name_time': {'S': 'file1_time1'}}, 
        {'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'inprogress'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable2'}, 'file_name_time': {'S': 'file2_time2'}}, 
        {'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'Fail'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable3'}, 'file_name_time': {'S': 'file3_time3'}}, 
        {'raw_ingestion_status': {'S': 'Pending'}, 'inhance_injection': {'S': 'Fail'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable4'}, 'file_name_time': {'S': 'file3_time4'}}
        ]
formatted_data = []
for entry in data:
    formatted_entry = {}
    for key, value in entry.items():
        formatted_entry[key] = value['S']
    formatted_data.append(formatted_entry)
   
# print(formatted_data)
for items in formatted_data:
    print(items)

{'raw_ingestion_status': 'Processed', 'inhance_injection': 'Completed', 'source_system': 'alfa', 'status': 'Active', 'table_name': 'SampleTable1', 'file_name_time': 'file1_time1'}
{'raw_ingestion_status': 'Processed', 'inhance_injection': 'inprogress', 'source_system': 'alfa', 'status': 'Active', 'table_name': 'SampleTable2', 'file_name_time': 'file2_time2'}
{'raw_ingestion_status': 'Processed', 'inhance_injection': 'Fail', 'source_system': 'alfa', 'status': 'Active', 'table_name': 'SampleTable3', 'file_name_time': 'file3_time3'}
{'raw_ingestion_status': 'Pending', 'inhance_injection': 'Fail', 'source_system': 'alfa', 'status': 'Active', 'table_name': 'SampleTable4', 'file_name_time': 'file3_time4'}


In [1]:
list_table = ["critical_process", "non_critical_process"]
for table_name in list_table:
    print(table_name)

critical_process
non_critical_process


{'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'Completed'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable1'}}


<class 'list'>
<class 'list'>
[{'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'Completed'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable1'}, 'file_name_time': {'S': 'file1_time1'}}, {'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'inprogress'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable2'}, 'file_name_time': {'S': 'file2_time2'}}, {'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'Fail'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable3'}, 'file_name_time': {'S': 'file3_time3'}}, {'raw_ingestion_status': {'S': 'Pending'}, 'inhance_injection': {'S': 'Fail'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable4'}, 'file_name_time': {'S': 'file3_time4'}}]


AttributeError: 'list' object has no attribute 'values'

CSV file created: output.csv


In [ ]:
[{'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'Completed'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable1'}, 'file_name_time': {'S': 'file1_time1'}}, {'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'inprogress'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable2'}, 'file_name_time': {'S': 'file2_time2'}}, {'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'Fail'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable3'}, 'file_name_time': {'S': 'file3_time3'}}, {'raw_ingestion_status': {'S': 'Pending'}, 'inhance_injection': {'S': 'Fail'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable4'}, 'file_name_time': {'S': 'file3_time4'}}]
i have this kind of data and i want to convert it into the structure format such that 

raw_ingestion_status	inhance_injection	source_system	status	table_name	file_name_time
Processed           	Completed	            alfa	    Active	SampleTable1	file1_time1
Processed           	inprogress          	alfa    	Active	SampleTable2	file2_time2

In [36]:
import pandas as pd

data = [{'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'Completed'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable1'}, 'file_name_time': {'S': 'file1_time1'}}, {'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'inprogress'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable2'}, 'file_name_time': {'S': 'file2_time2'}}, {'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'Fail'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable3'}, 'file_name_time': {'S': 'file3_time3'}}, {'raw_ingestion_status': {'S': 'Pending'}, 'inhance_injection': {'S': 'Fail'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable4'}, 'file_name_time': {'S': 'file3_time4'}}]

# Extracting data into a list of dictionaries
formatted_data = [{key: value['S'] for key, value in entry.items()} for entry in data]

# Creating DataFrame
df = pd.DataFrame(formatted_data)

print(df)


  raw_ingestion_status inhance_injection source_system  status    table_name  \
0            Processed         Completed          alfa  Active  SampleTable1   
1            Processed        inprogress          alfa  Active  SampleTable2   
2            Processed              Fail          alfa  Active  SampleTable3   
3              Pending              Fail          alfa  Active  SampleTable4   

  file_name_time  
0    file1_time1  
1    file2_time2  
2    file3_time3  
3    file3_time4  


In [ ]:
[('raw_ingestion_status', 'Processed'), ('inhance_injection', 'Completed'), ('source_system', 'alfa'), ('status', 'Active'), ('table_name', 'SampleTable1'), ('file_name_time', 'file1_time1'),
 ('raw_ingestion_status', 'Processed'), ('inhance_injection', 'inprogress'), ('source_system', 'alfa'), ('status', 'Active'), ('table_name', 'SampleTable2'), ('file_name_time', 'file2_time2'),
 ('raw_ingestion_status', 'Processed'), ('inhance_injection', 'Fail'), ('source_system', 'alfa'), ('status', 'Active'), ('table_name', 'SampleTable3'), ('file_name_time', 'file3_time3'), 
 ('raw_ingestion_status', 'Pending'), ('inhance_injection', 'Fail'), ('source_system', 'alfa'), ('status', 'Active'), ('table_name', 'SampleTable4'), ('file_name_time', 'file3_time4')
 ]

In [ ]:
[([('raw_ingestion_status', 'Processed'), ('inhance_injection', 'Completed'), ('source_system', 'alfa'), ('status', 'Active'), ('table_name', 'SampleTable1'), ('file_name_time', 'file1_time1'), 
   ('raw_ingestion_status', 'Processed'), ('inhance_injection', 'inprogress'), ('source_system', 'alfa'), ('status', 'Active'), ('table_name', 'SampleTable2'), ('file_name_time', 'file2_time2'), 
   ('raw_ingestion_status', 'Processed'), ('inhance_injection', 'Fail'), ('source_system', 'alfa'), ('status', 'Active'), ('table_name', 'SampleTable3'), ('file_name_time', 'file3_time3'), 
   ('raw_ingestion_status', 'Pending'), ('inhance_injection', 'Fail'), ('source_system', 'alfa'), ('status', 'Active'), ('table_name', 'SampleTable4'), ('file_name_time', 'file3_time4')],)]